In [0]:
import io, os
import re as re
import zipfile as zipfile
import math
import numpy as np
import random
from operator import itemgetter
import itertools
import scipy.spatial.distance
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
stop_words = ['a', 'acaba', 'altı', 'ama', 'ancak', 'artık', 'asla', 'aslında', 'az', 'b', 'bana', 'bazen', 'bazı', 'bazıları', 'bazısı', 'belki', 'ben', 'beni', 'benim', 'beş', 'bile', 'bir', 'birçoğu', 'birçok', 'birçokları', 'biri', 'birisi', 'birkaç', 'birkaçı', 'birşey', 'birşeyi', 'biz', 'bize', 'bizi', 'bizim', 'böyle', 'böylece', 'bu', 'buna', 'bunda', 'bundan', 'bunu', 'bunun', 'burada', 'bütün', 'c', 'ç', 'çoğu', 'çoğuna', 'çoğunu', 'çok', 'çünkü', 'd', 'da', 'daha', 'de', 'değil', 'demek', 'diğer', 'diğeri', 'diğerleri', 'diye', 'dokuz', 'dolayı', 'dört', 'e', 'elbette', 'en', 'f', 'fakat', 'falan', 'felan', 'filan', 'g', 'gene', 'gibi', 'ğ', 'h', 'hâlâ', 'hangi', 'hangisi', 'hani', 'hatta', 'hem', 'henüz', 'hep', 'hepsi', 'hepsine', 'hepsini', 'her', 'her biri', 'herkes', 'herkese', 'herkesi', 'hiç', 'hiç kimse', 'hiçbiri', 'hiçbirine', 'hiçbirini', 'ı', 'i', 'için', 'içinde', 'iki', 'ile', 'ise', 'işte', 'j', 'k', 'kaç', 'kadar', 'kendi', 'kendine', 'kendini', 'ki', 'kim', 'kime', 'kimi', 'kimin', 'kimisi', 'l', 'm', 'madem', 'mı', 'mı', 'mi', 'mu', 'mu', 'mü', 'mü', 'n', 'nasıl', 'ne', 'ne kadar', 'ne zaman', 'neden', 'nedir', 'nerde', 'nerede', 'nereden', 'nereye', 'nesi', 'neyse', 'niçin', 'niye', 'o', 'on', 'ona', 'ondan', 'onlar', 'onlara', 'onlardan', 'onların', 'onların', 'onu', 'onun', 'orada', 'oysa', 'oysaki', 'ö', 'öbürü', 'ön', 'önce', 'ötürü', 'öyle', 'p', 'r', 'rağmen', 's', 'sana', 'sekiz', 'sen', 'senden', 'seni', 'senin', 'siz', 'sizden', 'size', 'sizi', 'sizin', 'son', 'sonra', 'ş', 'şayet', 'şey', 'şeyden', 'şeye', 'şeyi', 'şeyler', 'şimdi', 'şöyle', 'şu', 'şuna', 'şunda', 'şundan', 'şunlar', 'şunu', 'şunun', 't', 'tabi', 'tamam', 'tüm', 'tümü', 'u', 'ü', 'üç', 'üzere', 'v', 'var', 've', 'veya', 'veyahut', 'y', 'ya', 'ya da', 'yani', 'yedi', 'yerine', 'yine', 'yoksa', 'z', 'zaten', 'zira']

In [0]:
commentsW = {}
mytextzip = ''
path = '/content/drive/My Drive/film_yorumlari.zip'

emotions = {
    "1": "pozitif",
    "2": "tarafsiz",
    "3": "negatif"
}

with zipfile.ZipFile(path) as z:
    for zipinfo in z.infolist():
        if zipinfo.filename.endswith('.txt') and re.search('raw_texts', zipinfo.filename):
            with z.open(zipinfo) as f:
                textfile = io.TextIOWrapper(f, encoding='cp1254', newline='')
                for line in textfile:
                    if len(line.strip()): mytextzip += line.strip()

                filename = str(zipinfo.filename)
                parts = filename.split("/")
                hash = parts[-2]
                for key, emot in emotions.items():
                    if hash == emot:
                        hash = key
                        break
                parts = parts[-1].split(".")
                doc = parts[0]
                
                commentsW[(doc, hash)] = mytextzip
                mytextzip = ''

In [0]:
def Tokenizer(text, stop):
    text = (str(text)).lower()          # normalize
    words = re.findall(r'\w+', text)    # find all words in it

    if stop == 1:
        #remove stopwords, start last word to avoid index problems in loop
        for x in range(len(words)-1, -1, -1):
            word = str(words[x])
            if word in stop_words:
                words.pop(x)
            elif word.lower() in stop_words:
                words.pop(x)
    
    return words

In [0]:
def turner(dictionary):
    for key, comment in dictionary.items():
        dictionary[key] = Tokenizer(comment, 1)
    return dictionary

In [0]:
commentsW = turner(commentsW)

In [8]:
i = 0
for (doc, emot), words in commentsW.items():
    print(doc, emot, words)
    if i == 2:
        break
    i += 1

1 3 ['senariste', 'lafım', 'rain', 'man', 'filmini', 'taklit', 'ederek', 'senaryo', 'yazılmaz', 'yönetmene', 'lafım', 'filmin', 'sonunda', 'milleti', 'ağlatmak', 'türlü', 'şaklabanlıklara', 'girilmez', 'olarak', 'yağmur', 'sahnesini', 'millet', 'anlamaz', 'bak', 'itfaiye', 'hortumuyla', 'sağlıyorum', 'görün', 'gösterilmez', 'bunları', 'göremeyen', 'izleyicilerde', 'sakın', 'film', 'harika', 'demesin']
10 3 ['çıkış', 'zamanı', 'itibariyle', 'hassas', 'dönemden', 'geçtiği', 'ülkemiz', 'film', 'gişe', 'yaptı', 'nsanlar', 'milliyetcilik', 'duygularıyla', 'akın', 'ettiler', 'sinemalara', 'ahım', 'şahım', 'bi', 'film', 'konu', 'yok', 'olaylar', 'arası', 'bağlantı', 'yok', 'görsellik', 'yok', 'sahneler', 'uzundu', 'bari', 'sonu', 'güzel', 'bağlansın', 'umut', 'ettim', 'tam', 'fiyaskoydu']
11 3 ['eğer', 'birazcık', 'filmden', 'anlıyorsam', 'film', 'para', 'etmez', 'tam', 'hayal', 'kırıklığı', 'sadece', 'insanların', 'duygularını', 'sömürüp', 'paraları', 'cebe', 'atmak', 'yapılmış', 'bi', 'film

In [0]:
term_frequencies = {}       # { doc: {word: word frequency} }

for (doc, emot), words in commentsW.items():
    d = {}
    temp = []
    
    #term frequency
    for word in words:
        if word not in d:
            d[word] = 1
        else:
            d[word] += 1

    n = len(d)      # of words in doc
    for word, number in d.items():
        tf = number / n
        temp.append([word, tf])

    key = str(doc)+"."+str(emot)
    term_frequencies[key] = temp

In [10]:
i = 0
for doc, freq in term_frequencies.items():
    print(doc, freq)
    if i == 2:
        break
    i += 1

1.3 [['senariste', 0.02857142857142857], ['lafım', 0.05714285714285714], ['rain', 0.02857142857142857], ['man', 0.02857142857142857], ['filmini', 0.02857142857142857], ['taklit', 0.02857142857142857], ['ederek', 0.02857142857142857], ['senaryo', 0.02857142857142857], ['yazılmaz', 0.02857142857142857], ['yönetmene', 0.02857142857142857], ['filmin', 0.02857142857142857], ['sonunda', 0.02857142857142857], ['milleti', 0.02857142857142857], ['ağlatmak', 0.02857142857142857], ['türlü', 0.02857142857142857], ['şaklabanlıklara', 0.02857142857142857], ['girilmez', 0.02857142857142857], ['olarak', 0.02857142857142857], ['yağmur', 0.02857142857142857], ['sahnesini', 0.02857142857142857], ['millet', 0.02857142857142857], ['anlamaz', 0.02857142857142857], ['bak', 0.02857142857142857], ['itfaiye', 0.02857142857142857], ['hortumuyla', 0.02857142857142857], ['sağlıyorum', 0.02857142857142857], ['görün', 0.02857142857142857], ['gösterilmez', 0.02857142857142857], ['bunları', 0.02857142857142857], ['gör

In [0]:
idfs = {}            # {word: number} ---> {word: idf}

for doc, array in term_frequencies.items():
    for word, tf in array:
        if word not in idfs:
            idfs[word] = 1
        else:
            idfs[word] += 1

doc_total = len(term_frequencies)
for word, n in idfs.items():
    idf = ((1 + doc_total) / (n+1))
    idfs[word] = math.log(idf) + 1

In [12]:
i = 0
for word, freqdocs in idfs.items():
    print(word, freqdocs)
    if i == 2:
        break
    i += 1

senariste 4.970291913552122
lafım 4.970291913552122
rain 3.8716796248840124


In [0]:
# TF-IDF = TF * IDF
tfidf = {}                      #{doc: {word: tf-idf}}

for doc, array in term_frequencies.items():
    tfidf[doc] = {}
    for word, tf in array:
        TfIdf = tf * idfs[word]
        tfidf[doc][word] = TfIdf

In [14]:
i = 0
for doc, ti in tfidf.items():
    print(doc, ti)
    if i == 2:
        break
    i += 1

1.3 {'senariste': 0.1420083403872035, 'lafım': 0.284016680774407, 'rain': 0.11061941785382892, 'man': 0.11061941785382892, 'filmini': 0.13042362301268448, 'taklit': 0.11582860519079903, 'ederek': 0.1420083403872035, 'senaryo': 0.10239993006949231, 'yazılmaz': 0.1420083403872035, 'yönetmene': 0.13042362301268448, 'filmin': 0.06660670239819609, 'sonunda': 0.11061941785382892, 'milleti': 0.1420083403872035, 'ağlatmak': 0.1420083403872035, 'türlü': 0.11582860519079903, 'şaklabanlıklara': 0.1420083403872035, 'girilmez': 0.1420083403872035, 'olarak': 0.07923049532045436, 'yağmur': 0.10239993006949231, 'sahnesini': 0.1420083403872035, 'millet': 0.13042362301268448, 'anlamaz': 0.1420083403872035, 'bak': 0.1420083403872035, 'itfaiye': 0.1420083403872035, 'hortumuyla': 0.1420083403872035, 'sağlıyorum': 0.1420083403872035, 'görün': 0.13042362301268448, 'gösterilmez': 0.1420083403872035, 'bunları': 0.13042362301268448, 'göremeyen': 0.1420083403872035, 'izleyicilerde': 0.1420083403872035, 'sakın': 

In [0]:
vectorS = []
dictionary = {}

i = 0
for doc, tis in tfidf.items():
    vectorS.append([])
    # doc = "1.11"
    # tis = {'time': 0.009378893551214249, 'year': 0.02088227471955536,...}
    # doc = "1.12"
    # tis = {'time': 0.010133140235402807, 'year': 0.011280809245354205,...}
    for word, df in idfs.items():
        # word = "doubt"
        if word not in tis:
            vectorS[i].append(0)
        else:
            ti = tis[word]
            vectorS[i].append(ti)
    emot = doc.split(".")
    emot = emot[1]
    if emot not in dictionary:
        dictionary[emot] = []
    dictionary[emot].append(vectorS[i])
    i += 1

In [0]:
def splitTrainTest(dict, n):
    train = {}
    train_v = []
    test_v = []
    test_a = []

    for emot, vectors in dict.items():
        train[emot] = []
        for i, vector in enumerate(vectors):
            if i >= (len(vectors)-n):     # last n index to test
                test_v.append(vector)
                test_a.append(emot)
                continue
            train[emot].append(vector)
            train_v.append(vector)
    return train, train_v, test_v, test_a

In [0]:
def findPriorProbability(dict):
    d = {}
    t_docs = len(vectorS)
    for emot, docs in dictionary.items():
        e_docs = len(docs)
        priP = e_docs / t_docs
        d[emot] = priP
    return d

In [0]:
def calculateMeanVar(dict, train_v):
    m = {}
    v = {}
    for emot, docs in dict.items():
        m[emot] = []
        v[emot] = []
        docs = np.array(docs)
        mean = np.mean(docs, axis=0)
        var = np.var(docs, axis=0) + (1e-9 * np.var(train_v, axis = 0).max())
        m[emot].append(mean)
        v[emot].append(var)
    return m, v

In [0]:
def pred(train, test_v, priorPrs, means, vars):
    preds = []
    for vector in test_v:
        vector = np.array(vector)
        posteriors = []
        emots = [*train]
        for emot in emots:
            prior = np.log(priorPrs[emot])
            mean = means[emot]
            std = vars[emot]

            s = -(0.5 * np.sum(((vector - mean) ** 2) / std)) - (0.5 * np.sum(np.log(np.multiply((2 * np.pi), std))))
            posterior = prior + s
            posteriors.append(posterior)
        prdInx = np.argmax(posteriors)
        pred = emots[prdInx]
        preds.append(pred)
    return preds

In [0]:
def NaiveBayes(dict, n):
    train = {}
    train_v = []
    test_v = []
    test_a = []

    train, train_v, test_v, test_a = splitTrainTest(dict, n)
    print("Emotions:", len(train), "\nTrain data:", len(train_v), "\nNumber of prediction:", len(test_v))

    priorPrs = findPriorProbability(train)
    means, vars = calculateMeanVar(train, train_v)

    predictions = []
    predictions = pred(train, test_v, priorPrs, means, vars)
    print("predictions:",predictions)
    print("real:       ", test_a)

    print("accuracy:", accuracy_score(predictions, test_a))

In [0]:
#print(len(dictionary)) # 3    -> # of emotions
#print(len(vectorS))    # 105  -> # of total docs

In [0]:
from sklearn.naive_bayes import GaussianNB

def splitTrainTestforGaussianNB(dict, n):
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    for emot, vectors in dict.items():
        for i, vector in enumerate(vectors):
            if i >= len(vectors)-n:     # last index to test
                X_test.append(vector)
                y_test.append(emot)
                break
            X_train.append(vector)
            y_train.append(emot)
    return X_train, y_train, X_test, y_test

In [0]:
def sklearnGaussianNB(dict, n):
    X_train, y_train, X_test, y_test = splitTrainTestforGaussianNB(dictionary, n)

    gnb = GaussianNB()
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
    y_pred = np.array(y_pred).tolist()

    print(accuracy_score(y_pred, y_test))

In [24]:
NaiveBayes(dictionary, 5)

Emotions: 3 
Train data: 90 
Number of prediction: 15
predictions: ['3', '1', '3', '3', '3', '1', '1', '1', '1', '1', '3', '3', '1', '2', '2']
real:        ['3', '3', '3', '3', '3', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2']
accuracy: 0.7333333333333333


In [25]:
sklearnGaussianNB(dictionary, 5)

0.6666666666666666
